In [1]:
from q_table_agent import QTableAgent

In [2]:
agent = QTableAgent.load_q_table('./experiments/cartpole/q_table_agent.csv')
result = agent.query_q_table([])
print(result)

Loading sparse Q-Table and configuration from ./experiments/cartpole/q_table_agent.csv...
Sparse Q-Table Details:
 - State Space Dimensions: 4
   Sizes: [16, 16, 16, 16]
 - Action Space Dimensions: 1
   Sizes: 2
 - Non-Zero Entries in Sparse Q-Table: 0
 - Total Possible Entries (Dense): 131072
 - Sparsity: 100.00%
Sparse Q-Table successfully loaded.
Sparse Q-Table Details:
 - State Space Dimensions: 4
   Sizes: [16, 16, 16, 16]
 - Action Space Dimensions: 1
   Sizes: 2
 - Non-Zero Entries in Sparse Q-Table: 6528
 - Total Possible Entries (Dense): 131072
 - Sparsity: 95.02%
      State_0_Index  State_1_Index  State_2_Index  State_3_Index  \
0                 7              7              8              7   
1                 7              8              8              6   
2                 7              8              8              5   
3                 7              8              7              6   
4                 7              7              7              7   
...         

In [3]:
filters = [
    {"field": "State_0_Value", "operator": ">", "value": 1},
    {"field": "State_1_Value", "operator": "<", "value": 0}
]
result = agent.query_q_table(filters)
print(result)


Empty DataFrame
Columns: [State_0_Index, State_1_Index, State_2_Index, State_3_Index, State_0_Value, State_1_Value, State_2_Value, State_3_Value, Action_0_Index, Q_Value, Visit_Count]
Index: []


In [4]:
result = agent.query_q_table([])
df = QTableAgent.compute_action_probabilities(result, strategy="greedy", epsilon=0.1)
print(df)
df = QTableAgent.compute_action_probabilities(result, strategy="softmax", temperature=1.0)
print(df)

      State_0_Index  State_1_Index  State_2_Index  State_3_Index  \
0                 0              0              7             13   
1                 0              0              8             14   
2                 0              0              9             12   
3                 0              0              9             13   
4                 0              0              9             14   
...             ...            ...            ...            ...   
3964              9              5              9             12   
3965              9              6              8             10   
3966              9              6              8             11   
3967              9              7              7              9   
3968              9              8              7              8   

      State_0_Value  State_1_Value  State_2_Value  State_3_Value  \
0             -2.40      -2.000000      -0.016667       1.466667   
1             -2.40      -2.000000       0.0166

In [5]:
from itertools import combinations

# List of all state features
state_features = ["State_0_Index", "State_1_Index", "State_2_Index", "State_3_Index"]

# Iterate over all possible combinations of features
for r in range(1, len(state_features) + 1):  # r is the number of features in each combination
    for feature_combination in combinations(state_features, r):
        # Compute mutual information for the current feature combination
        mi = QTableAgent.compute_mutual_information(
            df, list(feature_combination), "Action_0_Index", use_visit_count=True
        )
        # Print the results with detailed explanation
        print(f"Mutual Information for Features {feature_combination} with Action_0_Index: {mi}")

# Compute mutual information for all features as values
mi = QTableAgent.compute_mutual_information(
    df, ["State_0_Value", "State_1_Value", "State_2_Value", "State_3_Value"], "Action_0_Index", use_visit_count=True
)
print(f"Mutual Information for All Features as Values with Action_0_Index: {mi}")

Mutual Information for Features ('State_0_Index',) with Action_0_Index: 0.0019058441202399743
Mutual Information for Features ('State_1_Index',) with Action_0_Index: 0.019959984576330577
Mutual Information for Features ('State_2_Index',) with Action_0_Index: 0.1262079848463312
Mutual Information for Features ('State_3_Index',) with Action_0_Index: 0.11098679447405083
Mutual Information for Features ('State_0_Index', 'State_1_Index') with Action_0_Index: 0.07286285412985025
Mutual Information for Features ('State_0_Index', 'State_2_Index') with Action_0_Index: 0.16101371092841593
Mutual Information for Features ('State_0_Index', 'State_3_Index') with Action_0_Index: 0.16427904154778072
Mutual Information for Features ('State_1_Index', 'State_2_Index') with Action_0_Index: 0.26696843959201927
Mutual Information for Features ('State_1_Index', 'State_3_Index') with Action_0_Index: 0.1690799839104321
Mutual Information for Features ('State_2_Index', 'State_3_Index') with Action_0_Index: 0.3

In [6]:
result = agent.query_q_table([])
df = QTableAgent.compute_action_probabilities(result, strategy="softmax", temperature=1.0)
print(df)

      State_0_Index  State_1_Index  State_2_Index  State_3_Index  \
0                 0              0              7             13   
1                 0              0              8             14   
2                 0              0              9             12   
3                 0              0              9             13   
4                 0              0              9             14   
...             ...            ...            ...            ...   
3964              9              5              9             12   
3965              9              6              8             10   
3966              9              6              8             11   
3967              9              7              7              9   
3968              9              8              7              8   

      State_0_Value  State_1_Value  State_2_Value  State_3_Value  \
0             -2.40      -2.000000      -0.016667       1.466667   
1             -2.40      -2.000000       0.0166